# Preparation

## Import Libraries

In [92]:
!pip install -U transformers 

In [94]:
import numpy as np
import pandas as pd
from transformers import pipeline

## Connect google drive & Change directory

In [95]:
from google.colab import drive
import os


drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Colab Notebooks/팀 프로젝트/Comment_discrimination/')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Use KcELECTRA

## Load KcELECTRA

In [96]:
pipe_bias = pipeline('text-classification', model='beomi/beep-KcELECTRA-base-bias', device=0)

In [97]:
pipe_hate = pipeline('text-classification', model='beomi/beep-KcELECTRA-base-hate', device=0)

In [100]:
pipe_bias("안녕하세요")

[{'label': 'none', 'score': 0.9979233741760254}]

In [101]:
pipe_hate("안녕하세요")

[{'label': 'none', 'score': 0.9638350009918213}]

## Load data

In [98]:
df = pd.read_csv('data/test.csv')

In [102]:
df

,ID,title,comment
0,0,"류현경♥︎박성훈, 공개연애 4년차 애정전선 이상無..""의지 많이 된다""[종합]",둘다 넘 좋다~행복하세요
1,1,"""현금 유도+1인 1라면?""…'골목식당' 백종원, 초심 잃은 도시락집에 '경악' [종합]",근데 만원이하는 현금결제만 하라고 써놓은집 우리나라에 엄청 많은데
2,2,"입대 D-11' 서은광의 슬픈 멜로디..비투비, 눈물의 첫 체조경기장[콘서트 종합]",누군데 얘네?
3,3,"아이콘택트' 리쌍 길, 3년 전 결혼설 부인한 이유 공개…""결혼,출산 숨겼다""","쑈 하지마라 짜식아!음주 1번은 실수, 2번은 고의, 3번은 인간쓰레기다.슬금슬금 ..."
4,4,"구하라, 안검하수 반박 해프닝...""당당하다""vs""그렇게까지"" 설전 [종합]",안검하수 가지고 있는 분께 희망을 주고 싶은건가요? 수술하면 이렇게 자연스러워진다고...
...,...,...,...
506,506,"[N이슈] 최율, 조재현 성추행 의혹 폭로… 소속사 ""상황 파악 중""",얜 그냥 봐도 아니다 ㅋ 고소당하면 어마어마한 금액 물어줘야할껄?
507,507,"해투4' 이서진, 한지민 '대본 리딩 격리설' 해명…""날씨가 좋아서"" [SC컷]",대박 게스트... 꼭 봐야징~ 컨셉이 바뀌니깐 재미지넹
508,508,"[SS인터뷰①]박민영 ""'김비서' 행복했다..열애설엔 당당..미소였으니까""",성형으로 다 뜯어고쳐놓고 예쁜척. 성형 전 니 얼굴 다 알고있다. 순자처럼 된장냄새...
509,509,"[POP이슈]""사실무근"" 'SKY캐슬' 측 '위올라이' 표절설 부인→여전히 '핫'(종합)",분위기는 비슷하다만 전혀다른 전개던데 무슨ㅋㅋㄱ 우리나라사람들은 분위기만 비슷하면 ...


## Make discrimination

In [103]:
df['bias'] = df['comment'].map(lambda x: pipe_bias(x)[0]['label'])

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:978: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,


In [104]:
df['hate'] = df['comment'].map(lambda x: pipe_hate(x)[0]['label'])

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:978: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,


### Counts labels

In [105]:
df.bias.value_counts()

none      338
gender    107
others     66
Name: bias, dtype: int64

In [106]:
df.hate.value_counts() # 과제에서 사용하지않는 offensive label이 존재함

offensive    186
none         181
hate         144
Name: hate, dtype: int64

### Change 'offensive' label to 'hate' label

In [107]:
LABEL_DIC = {
    'none': 'none',
    'offensive': 'hate',
    'hate': 'hate',
}

In [108]:
df['hate'] = df['hate'].map(lambda x: LABEL_DIC[x])

In [109]:
df.hate.value_counts()

hate    330
none    181
Name: hate, dtype: int64

# Save Result

In [110]:
result = df[['ID', 'bias', 'hate']]
result

,ID,bias,hate
0,0,none,none
1,1,none,none
2,2,none,hate
3,3,none,hate
4,4,others,hate
...,...,...,...
506,506,none,none
507,507,none,none
508,508,others,hate
509,509,others,hate


In [111]:
result.to_csv('kcelectra.csv', index=None)